## Experiment design

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

from policy import NewPolicy, RandomPolicy
import simulator


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [49]:
# For changes in policy.py and simulator.py to be automatically reloaded without restarting the kernel
%load_ext autoreload
%autoreload 1
%aimport policy, simulator

In [3]:
treat_data = pd.read_csv("treatment_features.csv",header=None)
action_data = pd.read_csv("treatment_actions.csv",header=None)
outcome_data = pd.read_csv("treatment_outcomes.csv",header=None)

symptom_names = ['Covid-Recovered', 'Covid-Positive', 'No-Taste/Smell', 'Fever', 'Headache', 
                  'Pneumonia', 'Stomach', 'Myocarditis', 'Blood-Clots', 'Death','Age', 'Gender', 'Income']
cols = ( symptom_names +
         [f'Gene_{i+1:03}' for i in range(128)] +
         ['Asthma', 'Obesity', 'Smoking', 'Diabetes', 'Heart disease', 'Hypertension',
         'Vacc_1', 'Vacc_2', 'Vacc_3'])

treat_data.columns = cols
outcome_data.columns = cols[:10]
action_data.columns = ['Treatment_1', 'Treatment_2']

## Estimaring the $P(y|x,a)$:

In [5]:
X = treat_data
y=Y = outcome_data
a = action_data 

X_tr,X_ts,Y_tr,Y_ts = train_test_split(X,Y,test_size=0.33,random_state=1)
X_tr.columns = X.columns
X_ts.columns = X.columns
Y_tr.columns = Y.columns
Y_ts.columns = Y.columns


2021-12-02 17:36:25,124 : INFO : sklearn.model_selection.train_test_split: running accelerated version on CPU
2021-12-02 17:36:25,993 : INFO : sklearn.model_selection.train_test_split: running accelerated version on CPU


In [9]:
class MLPModel(MLPClassifier):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def get_probabilities(self, features, action):
        features = features.assign(**{str(action):np.ones(features.shape[0])})

        return self.predict_proba(features)

In [10]:
pol = NewPolicy(2, ['Treatment_1', 'Treatment_2'], 
                MLPModel(activation='logistic', random_state=0))

Initialising policy with  2 actions
A = { ['Treatment_1', 'Treatment_2'] }


In [11]:
pol.observe(X, a, Y)

In [28]:
X1 = X[a['Treatment_1'] == 1]
X2 = X[a['Treatment_2'] == 1]

#y1 = pd.DataFrame([Y.iloc[i] for i in range(len(a)) if a.iloc[i][0] == 1])
#y2 = pd.DataFrame([Y.iloc[i] for i in range(len(a)) if a.iloc[i][1] == 1])

y1 = y[a['Treatment_1'] == 1]
y2 = y[a['Treatment_2'] == 1]

## Utility

In [19]:
def get_utility(features, action, outcome):
    """Here the utiliy is defined in terms of the outcomes obtained only, ignoring both the treatment and the previous condition.
    """
    """for i, symptom in enumerate(symptom_names):
    utility[I] = weights[symptom] * outcome[symptom] """
    
    utility = np.zeros(10)

    utility[0] -= 0
    utility[1] -= 0
    utility[2]  -= 0.1 * sum(outcome['No-Taste/Smell'])
    utility[3]  -= 0.1 * sum(outcome['Fever'])
    utility[4]  -= 0.1 * sum(outcome['Headache'])
    utility[5]  -= 0.5 * sum(outcome['Pneumonia'])
    utility[6]  -= 0.2 * sum(outcome['Stomach'])
    utility[7]  -= 0.5 * sum(outcome['Myocarditis'])
    utility[8]  -= 1.0 * sum(outcome['Blood-Clots'])
    utility[9]  -= 100.0 * sum(outcome['Death'])
    
    return utility

## Estimating:
$$\sum P(y|x,a) \; u(a,y)$$

In [20]:
u1 = get_utility(None,None,y1)
u2 = get_utility(None,None,y2)

In [21]:
y2_proba = train_pred(X2,y2)
y1_proba = train_pred(X1,y1)

NameError: name 'train_pred' is not defined

In [ ]:
expected_utility1 = np.array([u*y for u,y in zip(u1,y1_proba)])
expected_utility2 = np.array([u*y for u,y in zip(u2,y2_proba)])

In [22]:
y1_proba.T @ u1

NameError: name 'y1_proba' is not defined

In [23]:
y2_proba.T @ u2

NameError: name 'y2_proba' is not defined

In [24]:
sum(expected_utility1)/len(X1)

NameError: name 'expected_utility1' is not defined

In [25]:
sum(expected_utility2)/len(X2)

NameError: name 'expected_utility2' is not defined

### Defining the utility

In [26]:
#finding optimal action for given symptoms for each individual

optimal_action = np.zeros([X.shape[0],2])
for i in range(X.shape[0]):
    index = np.argwhere(np.array(X.iloc[i,2:10]) == np.ones(8)).flatten().tolist()
    
    if len(index) > 0:
        if np.sum(expected_utility1[index]) >= np.sum(expected_utility2[index]):
            optimal_action[i,0] = 1
        else:
            optimal_action[i,1] = 1

NameError: name 'expected_utility1' is not defined

## Bootstrap the data for:
Provide error bounds on the expected utility and explain how those were obtained.

## Expected utility for improved policy

In [15]:
X1_improved = pd.DataFrame([X.iloc[i] for i in range(len(optimal_action)) if optimal_action[i][0] == 1])
X2_improved = pd.DataFrame([X.iloc[i] for i in range(len(optimal_action)) if optimal_action[i][1] == 1])

In [276]:
#taken from "simulator.py" file

def treatment(X,p):
    A = np.random.uniform(size=[2,8])
    treatments = np.zeros([X.shape[0], 2])
    result = np.zeros([X.shape[0], 8])
    for t in range(X.shape[0]):
        treatments[t] = optimal_action[t]
        r = np.array(np.matrix(treatments[t]) * A).flatten()
        for k in range(8):
            result[t,k] = np.random.choice(2,p=[1-p[k],p[k]])
        ##print("X:", X[t,:self.n_symptoms] , "Y:",  result[t])
    return treatments, result

In [295]:
p_symptoms1 = np.zeros(8)
p_symptoms2 = np.zeros(8)
for i in range(8):
    p_symptoms1[i] = y1.iloc[:,i+2].mean()
    p_symptoms2[i] = y2.iloc[:,i+2].mean()

In [304]:
Y1_new = pd.DataFrame(treatment(X1_improved.iloc[:,2:10],y1_proba)[1], columns = symptom_names[2:10])
Y2_new = pd.DataFrame(treatment(X2_improved.iloc[:,2:10],y2_proba)[1], columns = symptom_names[2:10])

In [305]:
new_action = pd.DataFrame({'Treatment1':optimal_action[:,0],'Treatment2':optimal_action[:,1]})

In [306]:
#doing the same as above for computing the expected utility

u1_new = get_utility(None,None,Y1_new)
u2_new = get_utility(None,None,Y2_new)

y2_new_proba = train_pred(X2_improved.iloc[:,2:10],Y2_new)
y1_new_proba = train_pred(X1_improved.iloc[:,2:10],Y1_new)

In [307]:
new_expected_utility1 = y1_new_proba.T @ u1_new[2:]
new_expected_utility2 = y2_new_proba.T @ u2_new[2:]

In [310]:
new_expected_utility1/len(X1_improved)

-0.12802193583611057

In [12]:
new_expected_utility2/len(X2_improved)

NameError: name 'new_expected_utility2' is not defined

In [23]:
pop = simulator.Population(128, 3, 2)


In [136]:
vacc_names = [f'Vacc_{i}' for i in range(4)]
vaccine_policy = NewPolicy(4, vacc_names, 
                           MLPModel(activation='logistic', random_state=0)) # make sure to add -1 for 'no vaccine'
rand_policy = RandomPolicy(4, list(range(4)))

# Only generate the population once, and then reuse previous result
pol_X = pd.DataFrame(getattr(pop, 'X', pop.generate(10_000)), columns=X.columns)

# Add Vacc_0 to the features when the person is not vaccinated
vacc_features= X_tr.assign(Vacc_0 = (np.ones(X.shape[0]) - X.iloc[:, -3:].sum(axis=1)))

# Must observe something to fit the model to make get_action work
vaccine_policy.observe(vacc_features, [], Y_tr)

pol_A = pd.DataFrame(vaccine_policy.get_action(pol_X), columns=vacc_names)
rand_A = rand_policy.get_action(pol_X)

Initialising policy with  4 actions
A = { ['Vacc_0', 'Vacc_1', 'Vacc_2', 'Vacc_3'] }
Initialising policy with  4 actions
A = { [0, 1, 2, 3] }


In [202]:
pol_Y = pd.DataFrame(pop.vaccinate(np.arange(10_000), pol_A.iloc[:, -1:].to_numpy()), columns=Y.columns)
rand_Y = pop.vaccinate(np.arange(10_000), rand_A[:,-1:])

print(f"Utility before vaccination {vaccine_policy.get_utility(pol_X, [], pol_X.iloc[:,:10])}")
print(f"Utility after vaccination {vaccine_policy.get_utility(pol_X, pol_A, pol_Y)}")
print(f"Utility random vaccination policy {rand_policy.get_utility(pol_X, rand_A, rand_Y)}")

Utility before vaccination -1502.8800000000028
Utility after vaccination -6856.1304037796235
Utility random vaccination policy -10971.1


The utility after vaccination is a lot lower than it was before the vaccination, but that is because the vaccinate method only adds symptoms randomly, and does not remove any. We do see, however, that the new vaccination policy gives better utility than the random one. 

In [139]:
treatment_policy = NewPolicy(2, ['Treatment_1', 'Treatment_2'], 
                           MLPModel(activation='logistic', random_state=0)) # make sure to add -1 for 'no vaccine'
rand_treat_policy = RandomPolicy(2, list(range(2)))


treat_X = pd.DataFrame(pop.X, columns=X.columns)

# Must observe something to fit the model to make get_action work
treatment_policy.observe(X_tr, a, Y_tr)

Initialising policy with  2 actions
A = { ['Treatment_1', 'Treatment_2'] }
Initialising policy with  2 actions
A = { [0, 1] }


In [205]:
treat_act, treat_Y = pop.treatment(treat_X, treatment_policy)
treat_act_rand, rand_treat_Y = pop.treatment(treat_X, rand_treat_policy)

print(f"Utility before treatment {treatment_policy.get_utility(treat_X, [], treat_X.iloc[:, :10])}")
print(f"Utility after treatment {treatment_policy.get_utility(treat_X, pd.DataFrame(treat_act, columns=treatment_policy.action_set), pd.DataFrame(treat_Y, columns=Y.columns))}")
print(f"Utility random treatment policy {rand_treat_policy.get_utility(treat_X, treat_act_rand, rand_treat_Y)}")


Utility before treatment -1502.8800000000028
Utility after treatment -1413.6091145212563
Utility random treatment policy -4229.4


Here the actions chosen by the new treatment policy gives slightly better utility than before treatment, and much better than the utility from the random policy.